# Installments

In [ ]:
#https://github.com/beir-cellar/beir/blob/main/examples/retrieval/evaluation/dense/evaluate_dpr.py
!pip install beir

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd

# Connect To Google Drive And Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/project_data'

scifact


In [ ]:
!ls '/content/drive/MyDrive/project_data/scifact'

corpus.jsonl	    queries.jsonl  test.source_rl  train.source     train.target_rl  val.target
process_scifact.py  test.csv	   test.target	   train.source_rl  val.source	     val.target_rl
qrels		    test.source    train.csv	   train.target     val.source_rl


In [ ]:
data_directory = '/content/drive/MyDrive/project_data'

#Load Meta-Data

In [ ]:
corpus = pd.read_json(data_directory + '/scifact/corpus.jsonl', lines=True)

corpus

,_id,title,text,metadata
0,4983,Microstructural development of human newborn c...,Alterations of the architecture of cerebral wh...,{}
1,5836,Induction of myelodysplasia by myeloid-derived...,Myelodysplastic syndromes (MDS) are age-depend...,{}
2,7912,"BC1 RNA, the transcript from a master gene for...",ID elements are short interspersed elements (S...,{}
3,18670,The DNA Methylome of Human Peripheral Blood Mo...,DNA methylation plays an important role in bio...,{}
4,19238,The human myelin basic protein gene is include...,Two human Golli (for gene expressed in the oli...,{}
...,...,...,...,...
5178,195689316,Body-mass index and cause-specific mortality i...,BACKGROUND The main associations of body-mass ...,{}
5179,195689757,Targeting metabolic remodeling in glioblastoma...,A key aberrant biological difference between t...,{}
5180,196664003,Signaling architectures that transmit unidirec...,A signaling pathway transmits information from...,{}
5181,198133135,"Association between pre-diabetes, type 2 diabe...",AIMS Trabecular bone score (TBS) is a surrogat...,{}


In [ ]:
queries = pd.read_json(data_directory + '/scifact/queries.jsonl', lines=True)
queries

,_id,text,metadata
0,0,0-dimensional biomaterials lack inductive prop...,{}
1,2,1 in 5 million in UK have abnormal PrP positiv...,"{'13734012': [{'sentences': [4], 'label': 'CON..."
2,4,1-1% of colorectal cancer patients are diagnos...,{}
3,6,10% of sudden infant death syndrome (SIDS) dea...,{}
4,9,32% of liver transplantation programs required...,"{'44265107': [{'sentences': [15], 'label': 'SU..."
...,...,...,...
1104,1379,Women with a higher birth weight are more like...,"{'16322674': [{'sentences': [5], 'label': 'SUP..."
1105,1382,aPKCz causes tumour enhancement by affecting g...,"{'17755060': [{'sentences': [3], 'label': 'CON..."
1106,1385,cSMAC formation enhances weak ligand signalling.,"{'306006': [{'sentences': [4], 'label': 'SUPPO..."
1107,1389,mTORC2 regulates intracellular cysteine levels...,"{'23895668': [{'sentences': [2, 3], 'label': '..."


# Load Train and Test data and qrels from .csv files

In [ ]:
#train_data
df_train = pd.read_csv(data_directory + '/scifact/train.csv', sep='\t')

print(df_train.shape)
print(df_train['query'].apply(len).mean())
print(df_train['text'].apply(len).mean())
df_train

(919, 3)
88.82372143634385
1535.5070729053318


,qid,query,text
0,0,0-dimensional biomaterials lack inductive prop...,Nanotechnologies are emerging platforms that c...
1,2,1 in 5 million in UK have abnormal PrP positiv...,OBJECTIVES To carry out a further survey of ar...
2,4,1-1% of colorectal cancer patients are diagnos...,CONTEXT Medicare's reimbursement policy was ch...
3,6,10% of sudden infant death syndrome (SIDS) dea...,Despite declines in prevalence during the past...
4,9,32% of liver transplantation programs required...,ContextChronic hepatitis C is the leading caus...
...,...,...,...
914,1404,siRNA knockdown of A20 slows tumor progression...,OBJECTIVE A20 is a TNF-inducible primary respo...
915,1405,taa1 tar1 tar2 triple mutants in Arabidopsis d...,Plants have evolved a tremendous ability to re...
916,1406,β-sheet opening occurs during pleurotolysin po...,Membrane attack complex/perforin-like (MACPF) ...
917,1407,β1/Ketel is able to bind microtubules.,The microtubule (MT) cytoskeleton is required ...


In [ ]:
#train qrels
df_train_qrels = pd.read_csv(data_directory + '/scifact/qrels/train.tsv', sep='\t')

df_train_qrels

,query-id,corpus-id,score
0,0,31715818,1
1,2,13734012,1
2,4,22942787,1
3,6,2613775,1
4,9,44265107,1
...,...,...,...
914,1404,38355793,1
915,1405,10504681,1
916,1406,2617858,1
917,1407,8087082,1


In [ ]:
#test data
df_test = pd.read_csv(data_directory + '/scifact/test.csv', sep='\t')
print(df_test.shape)
print(df_test['query'].apply(len).mean())
print(df_test['text'].apply(len).mean())
df_test

(339, 3)
91.43952802359883
1520.1592920353983


,qid,query,text
0,1,0-dimensional biomaterials show inductive prop...,Nanotechnologies are emerging platforms that c...
1,3,"1,000 genomes project enables mapping of genet...",Genome-wide association studies (GWAS) have no...
2,5,1/2000 in UK have abnormal PrP positivity.,OBJECTIVES To carry out a further survey of ar...
3,13,5% of perinatal mortality is due to low birth ...,CONTEXT One key target of the United Nations M...
4,36,A deficiency of vitamin B12 increases blood le...,BACKGROUND Homocysteine is a risk factor for c...
...,...,...,...
334,1379,Women with a higher birth weight are more like...,"INTRODUCTION Various perinatal factors, includ..."
335,1382,aPKCz causes tumour enhancement by affecting g...,Tumor cells have high-energetic and anabolic n...
336,1385,cSMAC formation enhances weak ligand signalling.,T cell activation is predicated on the interac...
337,1389,mTORC2 regulates intracellular cysteine levels...,Mutations in cancer reprogram amino acid metab...


In [ ]:
#test qrels
df_test_qrels = pd.read_csv(data_directory + '/scifact/qrels/test.tsv', sep='\t')

df_test_qrels

,query-id,corpus-id,score
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


# Transform dataframes into correct formated dicts

In [ ]:
#Note that _id here refers to doc id.
new_corpus = {}
for i in range(len(corpus)):
    key = str(corpus['_id'][i])
    new_corpus[key] = {'text' : corpus['text'][i], 'title' : corpus['title'][i]}

In [ ]:
#Note that qid here refers to query id.
new_train_queries = {}
for i in range(len(df_train)):
    key = str(df_train['qid'][i])
    new_train_queries[key] = df_train['query'][i]

new_test_queries = {}
for i in range(len(df_test)):
    key = str(df_test['qid'][i])
    new_test_queries[key] = df_test['query'][i]

In [ ]:
#train_qrels
new_train_qrels = {}
for i in range(len(df_train_qrels)):
    query_key = str(df_train_qrels['query-id'][i])
    new_train_qrels[query_key] = {str(df_train_qrels['corpus-id'][i]) : int(df_train_qrels['score'][i])}

#test_qrels
new_test_qrels = {}
for i in range(len(df_test_qrels)):
    query_key = str(df_test_qrels['query-id'][i])
    new_test_qrels[query_key] = {str(df_test_qrels['corpus-id'][i]) : int(df_test_qrels['score'][i])}

# Training

In [ ]:
from sentence_transformers import losses, models, SentenceTransformer, util
#from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.train import TrainRetriever
import pathlib, os
import logging




from beir.retrieval import models
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
import random

In [ ]:
#https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models/
#Note: The DPR models perform comparabily bad.
#Use the above mentioned Multi-QA models to achieve the optimal performance.

#Issues with DPR, so therefore for now just random pre-trained model
#link to DPR: https://www.sbert.net/docs/pretrained-models/dpr.html



#### Provide any sentence-transformers or HF model
#model_name = "distilbert-base-uncased"
#word_embedding_model = models.Transformer(model_name, max_seq_length=350)
#pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

model_name = "msmarco-distilbert-base-v3"
model = SentenceTransformer(model_name)


In [ ]:
retriever = TrainRetriever(model=model, batch_size=16)

In [ ]:
#### Prepare training samples

train_samples = retriever.load_train(new_corpus, new_train_queries, new_train_qrels)
train_dataloader = retriever.prepare_train(train_samples, shuffle=True)

Adding Input Examples:   0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
#### Training SBERT with cosine-product
train_loss = losses.MultipleNegativesRankingLoss(model=retriever.model)
#### training SBERT with dot-product
# train_loss = losses.MultipleNegativesRankingLoss(model=retriever.model, similarity_fct=util.dot_score)

In [ ]:
#### Prepare dev evaluator
#ir_evaluator = retriever.load_ir_evaluator(dev_corpus, dev_queries, dev_qrels)

#### If no dev set is present from above use dummy evaluator
ir_evaluator = retriever.load_dummy_evaluator()

In [ ]:
#### Provide model save path
model_save_path = os.path.join(pathlib.Path('./sample_data').parent.absolute(), "output", "{}-v1-{}".format(model_name, "scifact"))
os.makedirs(model_save_path, exist_ok=True)

In [ ]:
#### Configure Train params
num_epochs = 25
evaluation_steps = 5000
warmup_steps = int(len(train_samples) * num_epochs / retriever.batch_size * 0.1)

In [ ]:
retriever.fit(train_objectives=[(train_dataloader, train_loss)],
                evaluator=ir_evaluator,
                epochs=num_epochs,
                output_path=model_save_path,
                warmup_steps=warmup_steps,
                evaluation_steps=evaluation_steps,
                use_amp=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

# Evaluation

In [ ]:
evaluator_model = DRES(models.SentenceBERT(model_save_path, batch_size=16))
evaluator = EvaluateRetrieval(evaluator_model, score_function="dot")

In [ ]:
results = evaluator.retrieve(new_corpus, new_test_queries)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
ndcg, _map, recall, precision = evaluator.evaluate(new_test_qrels, results, [10,100,1000])

In [ ]:
ndcg

{'NDCG@10': 0.73255, 'NDCG@100': 0.75012, 'NDCG@1000': 0.75429}

In [ ]:
_map

{'MAP@10': 0.68873, 'MAP@100': 0.6925, 'MAP@1000': 0.69264}

In [ ]:
recall

{'Recall@10': 0.87, 'Recall@100': 0.95, 'Recall@1000': 0.98333}

In [ ]:
precision

{'P@10': 0.087, 'P@100': 0.0095, 'P@1000': 0.00098}

# Extras

In [ ]:
#### Just some code to print debug information to stdout
#dataset = "nfcorpus"
#url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
#out_dir = os.path.join(pathlib.Path('./sample_data').parent.absolute(), "datasets")
#data_path = util.download_and_unzip(url, out_dir)

#### Provide the data_path where nfcorpus has been downloaded and unzipped
#corpus2, queries2, qrels2 = GenericDataLoader(data_path).load(split="train")